# Tarea Semanal 4
Diseñar un filtro pasabanda con las siguientes especificaciones:
1. ω0=2π 22 kHz
2. Q=5
3. Aproximación Chebyshev con ripple de 0,5 dB
4. T(fS1)=-16 dB para fS1=17 kHz
5. T(fS2)=-24 dB para fS2=36 kHz

## Consignas

1. Obtener la plantilla de diseño pasabanda normalizada
2. Obtener la función transferencia normalizada del prototipo pasabajo que satisfaga el requerimiento del filtro pasabanda.
3. Obtener la transferencia pasabanda normalizada
4. Implementar mediante secciones pasivas separadas por seguidores de tensión activos.
5. Activar las redes pasivas mediante la red propuesta aquí debajo y comprobar mediante simulación el comportamiento deseado.

Para la obtención de la plantilla de diseño normalizada adopto como frecuencia de normalización 22kHz y calculo las frecuencias de interés normalizadas:

In [2]:
import numpy as np

wo = 22e3
q = 5
fs1 = 17e3
fs2 = 36e3

bw = (2*np.pi*wo)/q
fs1n = fs1/wo
fs2n = fs2/wo
print('Ancho de banda del filtro: {:0.0f} rad/seg'.format(bw))
print('Frecuencia de corte 1 normalizada: {:.4f}'.format(fs1n))
print('Frecuencia de corte 2 normalizada: {:.4f}'.format(fs2n))

Ancho de banda del filtro: 27646 rad/seg
Frecuencia de corte 1 normalizada: 0.7727
Frecuencia de corte 2 normalizada: 1.6364


Para obtener la transferencia normalizada del prototipo pasabajos, aplico el núcleo de transformación:
$$K(s)=p=Q.\frac{s^2+w_o^2}{s.w_o}$$
Normalizo y paso al dominio de la frecuencia:
$$\Omega=\frac{k(jw)}{j}=Q.\frac{w^2-1}{w}$$

In [3]:
Omega1 = abs(q*(fs1n**2-1)/fs1n)
Omega2 = abs(q*(fs2n**2-1)/fs2n)

print('Frecuencia de corte 1 transformada: {:.4f}'.format(Omega1))
print('Frecuencia de corte 2 transformada: {:.4f}'.format(Omega2))

Frecuencia de corte 1 transformada: 2.6070
Frecuencia de corte 2 transformada: 5.1263


Me quedo con la más exigente, en este caso 2,607. Con esto ya tengo definida la plantilla del pasabajos. 

In [4]:
alfa_min = 24
alfa_max = 0.5

epsilon_2 = 10**(alfa_max/10)-1
epsilon = np.sqrt(epsilon_2)
n = np.ceil(np.arccosh((np.power(10, alfa_min * 0.1) - 1) / (np.power(10, alfa_max * 0.1) - 1)) / (2 * np.arccosh(Omega1)))

print('Epsilon: {:.4f}'.format(epsilon))
print('Epsilon al cuadrado: {:.4f}'.format(epsilon_2))
print('Orden del filtro: {:0.0f}'.format(n))

Epsilon: 0.3493
Epsilon al cuadrado: 0.1220
Orden del filtro: 3


Para orden 3, la transferencia Chebyshev es:
$$ \lvert T(jw)\rvert^2=\frac{1}{1+\varepsilon^2(4w^3-3w)^2}$$

$$ \lvert T(jw)\rvert^2=\frac{1}{16\varepsilon^2w^6-24\varepsilon^2w^4+9\varepsilon^2w^2+1}$$

Haciendo el reemplazo: $T(s).T(-s)=\lvert T(jw)\rvert^2\rfloor_{w=\frac{s}{j}}$

$$ T(s).T(-s)=\frac{1}{-16\varepsilon^2s^6-24\varepsilon^2s^4-9\varepsilon^2s^2+1}$$

$$ T(s).T(-s)=\frac{\frac{1}{-16\varepsilon^2}}{s^6+1,5s^4+0,5625s^2-\frac{1}{16\varepsilon^2}}$$

A continuación, obtengo las raíces del polinomio:

In [5]:
pol = [1, 0, 1.5, 0, 0.5625, 0, -(1)/(16*epsilon_2)]
np.set_printoptions(precision=3)
print(np.roots(pol))

[-0.313+1.022j -0.313-1.022j  0.313+1.022j  0.313-1.022j -0.626+0.j
  0.626+0.j   ]


Me quedo con las raíces correspondientes a $T(s)$

$$ T(s)=\frac{1,1424}{s^2+0,626.s+1,1424}\frac{0,626}{s+0,626}$$

Aplico a continuación el núcleo de transformación pasabanda: $K(s)=Q.\frac{s^2+1}{s}$

$$ T(s)=\frac{1,1424}{(Q.\frac{s^2+1}{s})^2+0,626.Q.\frac{s^2+1}{s}+1,1424}.\frac{0,626}{Q.\frac{s^2+1}{s}+0,626}$$

Haciendo desarrollo matemático, se llega a:

$$ T(s)=\frac{\frac{1,1424}{Q^2}s^2}{s^4+0,1252s^3+2,046s^2+0,1252s+1}.\frac{\frac{0,626}{Q}s}{s^2+0,1252s+1}$$

Calculo las raíces del polinomio de orden 4

In [6]:
pol = [1, 0.1252, 2.046, 0.1252, 1]
np.set_printoptions(precision=3)
print(np.roots(pol))

[-0.034+1.107j -0.034-1.107j -0.028+0.902j -0.028-0.902j]


$$ T(s)=\frac{\frac{1,2266}{Q}s}{s^2+0,069s+1,2266}.\frac{\frac{0,814}{Q}s}{s^2+0,056s+0,814}.\frac{\frac{0,626}{Q}s}{s^2+0,1252s+1} $$

A partir de las transferencias, se pueden obtener las estructuras pasivas asociadas LCR:

In [7]:
wo1_2 = 1.2266
wo1_q = 0.069

wo1 = np.sqrt(wo1_2)
q1 = wo1/wo1_q
k1 = (wo1/q1)*(q/wo1_2)

print('Parametros del 1er filtro:')
print('Wo1={:.4f} - Q1={:.4f} - k1={:.4f}'.format(wo1, q1, k1))

l1 = q1/wo1
c1 = 1/(wo1*q1)
r1 = 1

print()
print('Componentes normalizados del 1er filtro:')
print('L1={:.4f} - C1={:.4f} - R1={:.4f}'.format(l1, c1, r1))

Parametros del 1er filtro:
Wo1=1.1075 - Q1=16.0510 - k1=0.2813

Componentes normalizados del 1er filtro:
L1=14.4928 - C1=0.0563 - R1=1.0000


In [8]:
wo2_2 = 0.814
wo2_q = 0.056

wo2 = np.sqrt(wo2_2)
q2 = wo2/wo2_q
k2 = (wo2/q2)*(q/wo2_2)

print('Parametros del 2do filtro:')
print('Wo2={:.4f} - Q2={:.4f} - k2={:.4f}'.format(wo2, q2, k2))

l2 = q2/wo2
c2 = 1/(wo2*q2)
r2 = 1

print()
print('Componentes normalizados del 2do filtro:')
print('L2={:.4f} - C2={:.4f} - R2={:.4f}'.format(l2, c2, r2))

Parametros del 2do filtro:
Wo2=0.9022 - Q2=16.1111 - k2=0.3440

Componentes normalizados del 2do filtro:
L2=17.8571 - C2=0.0688 - R2=1.0000


In [9]:
wo3_2 = 1
wo3_q = 0.1252

wo3 = np.sqrt(wo3_2)
q3 = wo3/wo3_q
k3 = (wo3/q3)*(q/wo3_2)

print('Parametros del 3er filtro:')
print('Wo3={:.4f} - Q3={:.4f} - k3={:.4f}'.format(wo3, q3, k3))

l3 = q3/wo3
c3 = 1/(wo3*q3)
r3 = 1

print()
print('Componentes normalizados del 3er filtro:')
print('L3={:.4f} - C3={:.4f} - R3={:.4f}'.format(l3, c3, r3))

Parametros del 3er filtro:
Wo3=1.0000 - Q3=7.9872 - k3=0.6260

Componentes normalizados del 3er filtro:
L3=7.9872 - C3=0.1252 - R3=1.0000


Desnormalización de los componentes de los filtros:

In [10]:
norma_z = 10e3
norma_w = 2*np.pi*wo

l1 = (l1*norma_z)/norma_w
c1 = c1/(norma_z*norma_w)
r1_1 = r1*norma_z - k1*norma_z
r1_2 = r1*norma_z - r1_1

print('Componentes del 1er filtro:')
print('L1={:.4f} Hy - C1={:.3e} F - R1_1={:.4e} Ohm - R1_1={:.4e} Ohm'.format(l1, c1, r1_1, r1_2))

Componentes del 1er filtro:
L1=1.0485 Hy - C1=4.070e-11 F - R1_1=7.1873e+03 Ohm - R1_1=2.8127e+03 Ohm


In [11]:
l2 = (l2*norma_z)/norma_w
c2 = c2/(norma_z*norma_w)
r2_1 = r2*norma_z - k2*norma_z
r2_2 = r2*norma_z - r2_1

print('Componentes del 2do filtro:')
print('L2={:.4f} Hy - C2={:.3e} F - R2_1={:.4e} Ohm - R2_2={:.4e} Ohm'.format(l2, c2, r2_1, r2_2))

Componentes del 2do filtro:
L2=1.2918 Hy - C2=4.977e-11 F - R2_1=6.5602e+03 Ohm - R2_2=3.4398e+03 Ohm


In [12]:
l3 = (l3*norma_z)/norma_w
c3 = c3/(norma_z*norma_w)
r3_1 = r3*norma_z

print('Componentes del 3er filtro:')
print('L3={:.4f} Hy - C3={:.3e} F - R3={:.4e} Ohm'.format(l3, c3, r3_1))

Componentes del 3er filtro:
L3=0.5778 Hy - C3=9.057e-11 F - R3=1.0000e+04 Ohm


Para la verificación, se realizó la siguiente simulación en LTSpice:

<div>
<img src="img/circuito.PNG" alt="Drawing" style="width: 800px;"/>
</div>

A lo cual se obtuvieron los siguientes resultados:

<div>
<img src="img/simulacion.PNG" alt="Drawing" style="width: 800px;"/>
</div>

Se puede observar que la transferencia cumple con las condiciones establecidas en cuanto a formas, pero no en cuanto a magnitud. Se puede ver que nos encontramos 40dB por debajo de lo establecido.

Para activar las bobinas, se usará el siguiente circuito:


<div>
<img src="img/activador.png" alt="Drawing" style="width: 800px;"/>
</div>

Este circuito es equivalente a colocar una bobina a masa, por lo que cambia la topología de la primera propuesta por su circuito dual. Por lo tanto, se recalculan a continuación los componentes para este nuevo circuito:

In [24]:
c1 = q1/wo1
l1 = 1/(wo1*q1)
r1 = 1

print()
print('Componentes normalizados del 1er filtro:')
print('L1={:.4f} - C1={:.4f} - R1={:.4f}'.format(l1, c1, r1))

l1 = (l1*norma_z)/norma_w
c1 = c1/(norma_z*norma_w)
r1_1 = (r1*norma_z)/k1
r1_2 = (r1*norma_z)/(1-k1)

print('Componentes del 1er filtro:')
print('L1={:.3e} Hy - C1={:.3e} F - R1_1={:.4e} Ohm - R1_2={:.4e} Ohm'.format(l1, c1, r1_1, r1_2))


Componentes normalizados del 1er filtro:
L1=0.0563 - C1=14.4928 - R1=1.0000
Componentes del 1er filtro:
L1=4.070e-03 Hy - C1=1.048e-08 F - R1_1=3.5554e+04 Ohm - R1_2=1.3913e+04 Ohm


In [27]:
c2 = q2/wo2
l2 = 1/(wo2*q2)
r2 = 1

print()
print('Componentes normalizados del 2do filtro:')
print('L2={:.4f} - C2={:.4f} - R2={:.4f}'.format(l2, c2, r2))

l2 = (l2*norma_z)/norma_w
c2 = c2/(norma_z*norma_w)
r2_1 = (r2*norma_z)/k2
r2_2 = (r2*norma_z)/(1-k2)

print('Componentes del 2do filtro:')
print('L2={:.3e} Hy - C2={:.3e} F - R2_1={:.4e} Ohm - R2_2={:.4e} Ohm'.format(l2, c2, r2_1, r2_2))


Componentes normalizados del 2do filtro:
L2=0.0688 - C2=17.8571 - R2=1.0000
Componentes del 2do filtro:
L2=4.977e-03 Hy - C2=1.292e-08 F - R2_1=2.9071e+04 Ohm - R2_2=1.5243e+04 Ohm


In [30]:
c3 = q3/wo3
l3 = 1/(wo3*q3)
r3= 1

print()
print('Componentes normalizados del 3er filtro:')
print('L3={:.4f} - C3={:.4f} - R3={:.4f}'.format(l3, c3, r3))

l3 = (l3*norma_z)/norma_w
c3 = c3/(norma_z*norma_w)
r3_1 = r3*norma_z

print('Componentes del 3er filtro:')
print('L3={:.3e} Hy - C3={:.3e} F - R3={:.4e} Ohm'.format(l3, c3, r3_1))


Componentes normalizados del 3er filtro:
L3=0.1252 - C3=7.9872 - R3=1.0000
Componentes del 3er filtro:
L3=9.057e-03 Hy - C3=5.778e-09 F - R3=1.0000e+04 Ohm


Conociendo los valores de inductancia deseados, calculamos los valores de los componentes que permiten la activación de los inductores: $Leq=R^2.C$

In [37]:
c = 1e-9

r_l1 = np.sqrt(l1/c)
r_l2 = np.sqrt(l2/c)
r_l3 = np.sqrt(l3/c)

print('En todos los casos usamos C={:.0e}'.format(c))
print('Resistencias para activar inductor 1: R={:.3e}'.format(r_l1))
print('Resistencias para activar inductor 2: R={:.3e}'.format(r_l2))
print('Resistencias para activar inductor 3: R={:.3e}'.format(r_l3))

En todos los casos usamos C=1e-09
Resistencias para activar inductor 1: R=2.017e+03
Resistencias para activar inductor 2: R=2.231e+03
Resistencias para activar inductor 3: R=3.010e+03


Ahora ya tenemos los valores de todos los componentes, por lo tanto procedemos a armar el circuito en LTSpice y simularlo:

<div>
<img src="img/activado.PNG" alt="Drawing" style="width: 800px;"/>
</div>

A lo que se obtuvo la siguiente transferencia:

<div>
<img src="img/simu_activado.PNG" alt="Drawing" style="width: 800px;"/>
</div>

Se obtuvieron los mismos resultados que en la simulación previa, con la misma atenuación de 40dB que se había observado anteriormente.

<!-- $$ T(s)=\frac{\frac{1}{4\varepsilon}}{s^3+1,252.s^2+1,5343.s+0,7151}$$ -->